## Standard x86 data

In [4]:
import pandas as pd
import numpy as np
import os
from os.path import join, pardir
DATA_DIR = join(pardir, pardir, 'data', 'write_new_algorithm')


dfs = []
dfs2 = []

configuration = 'persistent'
for mem in [512, 2048]:

    for size in [4, 1024, 65536, 131072, 256000]:
        df = pd.read_csv(join(DATA_DIR, f'persistent_{size}_{mem}_processed.csv'), index_col=0)
        df['size'] = size / 1024
        df['system'] = configuration
        dfs.append(df)


        df = pd.read_csv(join(DATA_DIR, f'persistent_{size}_{mem}.csv'), index_col=0)
        #df = df.groupby(['memory', 'size']).head(100).reset_index(drop=True)
        df['size'] = size / 1024
        #df['data'] /= 1000.0
        #df.loc[df['op'] == 'set_data', 'data'] /= 1000
        dfs2.append(df)
        
configuration = 'keyvalue'
for mem in [512, 2048]:

    for size in [4, 128, 256, 512, 1024,2048,4096]:
        df = pd.read_csv(join(DATA_DIR, 'keyvalue', f'persistent_{size}_{mem}_processed.csv'), index_col=0)
        df['size'] = size / 1024
        df['system'] = configuration
        dfs.append(df)


        df = pd.read_csv(join(DATA_DIR, 'keyvalue', f'persistent_{size}_{mem}.csv'), index_col=0)
        #df = df.groupby(['memory', 'size']).head(100).reset_index(drop=True)
        df['size'] = size / 1024
        #df['data'] /= 1000.0
        #df.loc[df['op'] == 'set_data', 'data'] /= 1000
        dfs2.append(df)

        
x86_data = pd.concat(dfs)
#data['system'] = 'keyvalue'
x86_data_client = pd.concat(dfs2)
#data_client['system'] = 'persistent'


### ARM Data

In [5]:
import pandas as pd
import numpy as np
import os
from os.path import join, pardir
DATA_DIR = join(pardir, pardir, 'data', 'write_new_algorithm')


dfs = []
dfs2 = []
for configuration in ['dynamo', 'persistent']:

    for mem in [512, 2048]:

        for size in [4, 1024, 65536, 131072, 256000]:
            df = pd.read_csv(join(DATA_DIR, 'arm', f'{configuration}_sqs', f'arm_{configuration}_{size}_{mem}_processed.csv'), index_col=0)
            #print(join(DATA_DIR, 'arm', f'{configuration}_sqs', f'arm_{configuration}_{size}_{mem}_processed.csv'))
            df['size'] = size / 1024
            df['system'] = configuration if configuration == 'persistent' else 'keyvalue'
            dfs.append(df)
            #print(df.head())


            df = pd.read_csv(join(DATA_DIR, 'arm', f'{configuration}_sqs', f'arm_{configuration}_{size}_{mem}.csv'), index_col=0)
            #df = df.groupby(['memory', 'size']).head(100).reset_index(drop=True)
            df['size'] = size / 1024
            #df['data'] /= 1000.0
            #df.loc[df['op'] == 'set_data', 'data'] /= 1000
            dfs2.append(df)

        
arm_data = pd.concat(dfs)
arm_data.to_csv('test.csv')
#data['system'] = 'keyvalue'
arm_data_client = pd.concat(dfs2)
#data_client['system'] = 'persistent'


### Performance

In [15]:
arm_data_summary = arm_data.groupby(['system', 'function', 'size', 'memory'])['time'].median().reset_index()
arm_data_mean = arm_data.groupby(['system', 'function', 'size', 'memory'])['time'].mean().reset_index()
arm_data_summary['mean'] = arm_data_mean['time']
print(arm_data_summary.to_markdown())

|    | system     | function    |         size |   memory |    time |     mean |
|---:|:-----------|:------------|-------------:|---------:|--------:|---------:|
|  0 | keyvalue   | distributor |   0.00390625 |      512 |  35.25  |  37.4256 |
|  1 | keyvalue   | distributor |   0.00390625 |     2048 |  24.725 |  25.5161 |
|  2 | keyvalue   | distributor |   1          |      512 |  42.88  |  43.2512 |
|  3 | keyvalue   | distributor |   1          |     2048 |  31.725 |  33.069  |
|  4 | keyvalue   | distributor |  64          |      512 |  62.555 |  66.7355 |
|  5 | keyvalue   | distributor |  64          |     2048 |  37.51  |  41.5539 |
|  6 | keyvalue   | distributor | 128          |      512 |  77.67  |  82.3625 |
|  7 | keyvalue   | distributor | 128          |     2048 |  46.96  |  55.7212 |
|  8 | keyvalue   | distributor | 250          |      512 |  73.735 |  76.8126 |
|  9 | keyvalue   | distributor | 250          |     2048 |  61.305 |  65.6112 |
| 10 | keyvalue   | writer  

In [14]:
x86_data_summary = x86_data.groupby(['system', 'function', 'size', 'memory'])['time'].median().reset_index()
x86_data_mean = x86_data.groupby(['system', 'function', 'size', 'memory'])['time'].mean().reset_index()
x86_data_summary['mean'] = x86_data_mean['time']
print(x86_data_summary.to_markdown())

|    | system     | function    |         size |   memory |    time |     mean |
|---:|:-----------|:------------|-------------:|---------:|--------:|---------:|
|  0 | keyvalue   | distributor |   0.00390625 |      512 |  26.47  |  29.3676 |
|  1 | keyvalue   | distributor |   0.00390625 |     1024 |  25.04  |  26.9786 |
|  2 | keyvalue   | distributor |   0.00390625 |     2048 |  25.8   |  28.5561 |
|  3 | keyvalue   | distributor |   0.125      |      512 |  27.595 |  29.5461 |
|  4 | keyvalue   | distributor |   0.125      |     1024 |  25.35  |  27.5542 |
|  5 | keyvalue   | distributor |   0.125      |     2048 |  29.82  |  33.5219 |
|  6 | keyvalue   | distributor |   0.25       |      512 |  28.935 |  32.5057 |
|  7 | keyvalue   | distributor |   0.25       |     1024 |  23.275 |  26.5605 |
|  8 | keyvalue   | distributor |   0.25       |     2048 |  33.99  |  36.9281 |
|  9 | keyvalue   | distributor |   0.5        |      512 |  29.36  |  31.6895 |
| 10 | keyvalue   | distribu

### Hybrid Range

In [16]:
subset = arm_data_summary.loc[arm_data_summary['system'] == 'persistent']
x86_subset = x86_data_summary.loc[x86_data_summary['system'] == 'persistent']
#print(x86_subset)

for mem in [512, 2048]:

    for size in [4, 1024, 65536, 131072, 256000]:
        
        arm_time = subset.loc[(subset['size'] == size/1024) & (subset['memory'] == mem)]
        arm_writer = arm_time[arm_time['function'] == 'writer']['time']
        if len(arm_writer.values) > 0:
            arm_writer = arm_writer.values[0]
        else:
            continue
        
        x86_time = x86_subset.loc[(x86_subset['size'] == size/1024) & (x86_subset['memory'] == mem)]
        x86_writer = x86_time[x86_time['function'] == 'writer']['time'].values[0]
        
        x86_cost = x86_writer * 1.66667 * 10**-5
        arm_cost = arm_writer * 1.33334 * 10**-5

        print(mem, size, 'writer', arm_writer, x86_writer, (1-arm_writer/x86_writer)*100.0, (1-arm_cost/x86_cost)*100.0)
        
print('--------------')
for mem in [512, 2048]:

    for size in [4, 1024, 65536, 131072, 256000]:
        arm_time = subset.loc[(subset['size'] == size/1024) & (subset['memory'] == mem)]
        arm_writer = arm_time[arm_time['function'] == 'distributor']['time']
        if len(arm_writer.values) > 0:
            arm_writer = arm_writer.values[0]
        else:
            continue
        
        x86_time = x86_subset.loc[(x86_subset['size'] == size/1024) & (x86_subset['memory'] == mem)]
        x86_writer = x86_time[x86_time['function'] == 'distributor']['time'].values[0]

        print(mem, size, 'distributor', arm_writer, x86_writer, (1-arm_writer/x86_writer)*100.0)
        

512 4 writer 28.05 34.555 18.825061496165528 35.059854377469655
512 1024 writer 28.325 31.07 8.83488896041198 27.067692372500684
512 65536 writer 64.16499999999999 72.33 11.288538642333757 29.03061800678557
512 131072 writer 103.44 122.83 15.786045754294554 32.6286344903497
512 256000 writer 180.65 223.95 19.334672918062058 35.46754473805185
2048 4 writer 29.35 31.81 7.733417164413703 26.186512292175046
2048 1024 writer 32.665 31.384999999999998 -4.0783813923849 16.73704509847631
2048 65536 writer 43.89 47.76 8.103015075376884 26.48219150797879
2048 131072 writer 58.16 66.275 12.244436061863462 29.795338236558557
2048 256000 writer 87.17 102.53 14.980981176241103 31.984580895755798
--------------
512 4 distributor 79.24 60.5 -30.97520661157025
512 1024 distributor 92.05 60.59 -51.922759531275766
512 65536 distributor 103.51 91.09 -13.634866615435293
512 131072 distributor 144.265 74.48 -93.69629430719652
512 256000 distributor 161.31 140.65 -14.68894418769997
2048 4 distributor 69.81 6

In [21]:
import math
lambda_const = 1.66667 * 10**-5
arm_const = 0.0000133334

def func_cost(memory, length):
    return memory/1024.0 * length/1000.0 * lambda_const + 0.2 * 10**-6
def arm_cost(memory, length):
    return memory/1024.0 * length/1000.0 * arm_const + 0.2 * 10**-6


#### Cost with invocations

In [22]:
subset = arm_data_summary.loc[arm_data_summary['system'] == 'persistent']
x86_subset = x86_data_summary.loc[x86_data_summary['system'] == 'persistent']
#print(x86_subset)


print('-----WRITER--------')

for mem in [512, 2048]:

    for size in [4, 1024, 65536, 131072, 256000]:
        
        arm_time = subset.loc[(subset['size'] == size/1024) & (subset['memory'] == mem)]
        arm_writer = arm_time[arm_time['function'] == 'writer']['time']
        if len(arm_writer.values) > 0:
            arm_writer = arm_writer.values[0]
        else:
            continue
        
        x86_time = x86_subset.loc[(x86_subset['size'] == size/1024) & (x86_subset['memory'] == mem)]
        x86_writer = x86_time[x86_time['function'] == 'writer']['time'].values[0]

        print(mem, size, 'writer', arm_writer, x86_writer, (1-arm_writer/x86_writer)*100.0, 1- arm_cost(mem, arm_writer)/func_cost(mem, x86_writer))
        

print('-----DISTRIBUTOR--------')

for mem in [512, 2048]:

    for size in [4, 1024, 65536, 131072, 256000]:
        
        arm_time = subset.loc[(subset['size'] == size/1024) & (subset['memory'] == mem)]
        arm_writer = arm_time[arm_time['function'] == 'distributor']['time']
        if len(arm_writer.values) > 0:
            arm_writer = arm_writer.values[0]
        else:
            continue
        
        x86_time = x86_subset.loc[(x86_subset['size'] == size/1024) & (x86_subset['memory'] == mem)]
        x86_writer = x86_time[x86_time['function'] == 'distributor']['time'].values[0]

        print(mem, size, 'distributor', arm_writer, x86_writer, (1-arm_writer/x86_writer)*100.0, 1- arm_cost(mem, arm_writer)/func_cost(mem, x86_writer))
        

-----WRITER--------
512 4 writer 28.05 34.555 18.825061496165528 0.20689851611721544
512 1024 writer 28.325 31.07 8.83488896041198 0.15271362539268785
512 65536 writer 64.16499999999999 72.33 11.288538642333757 0.2179783708839289
512 131072 writer 103.44 122.83 15.786045754294554 0.2729535166262964
512 256000 writer 180.65 223.95 19.334672918062058 0.32034515756162085
2048 4 writer 29.35 31.81 7.733417164413703 0.22031029367519483
2048 1024 writer 32.665 31.384999999999998 -4.0783813923849 0.14050884820809406
2048 65536 writer 43.89 47.76 8.103015075376884 0.23526595028639197
2048 131072 writer 58.16 66.275 12.244436061863462 0.27321845305975845
2048 256000 writer 87.17 102.53 14.980981176241103 0.3021634056792778
-----DISTRIBUTOR--------
512 4 distributor 79.24 60.5 -30.97520661157025 -0.03422712211264156
512 1024 distributor 92.05 60.59 -51.922759531275766 -0.15427625398751066
512 65536 distributor 103.51 91.09 -13.634866615435293 0.07195894543852066
512 131072 distributor 144.265 74